In [2]:
#from data_utils.snowflake_manager import SnowflakeAdapter
from dotenv import load_dotenv
import os
import json
import pandas as pd

In [23]:
import sys
sys.path.insert(0, '/Users/michaelfelipepedrazacardenas/Documents/GitHub/cobre_data_science/utils/libs/cobre_ds_utils/src')

from cobre_ds_utils.adapters.snowflake_adapter import SnowflakeAdapter

In [10]:
load_dotenv('/Users/michaelfelipepedrazacardenas/Documents/GitHub/cobre_data_science/projects/tests/mms_billing/local.env')

True

In [9]:
load_dotenv('.env')

False

In [11]:
from dotenv import load_dotenv
import os
from cobre_ds_utils.adapters.snowflake_adapter import SnowflakeAdapter

# Cargar el archivo
user = os.getenv('account')
account = os.getenv('warehouse')
warehouse = os.getenv('database')
database = os.getenv('schema')
authenticator = os.getenv('authenticator')
print(user, account, warehouse, database, authenticator)

snowflake_adapter = SnowflakeAdapter(user="michelp@cobre.co", account="xyb94886.us-east-1", warehouse="COMPUTE_WH", database="COBRE_GOLD_DB")


xyb94886.us-east-1 COBRE_GOLD_DB COBRE_GOLD_DB.DATAWAREHOUSE None externalbrowser
Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://accounts.google.com/o/saml2/idp?idpid=C03jwcx66&SAMLRequest=lZJfb9owFMW%2FSuQ9J3bSDopFQCmsKxPtaIHuz5tjGzB17Mx2COzTzwmN1D20Uh8sWfbv%2BJ7rc4fjYyGDAzdWaJWCOEIg4IpqJtQ2BevVTXgFAuuIYkRqxVNw4haMR0NLClnirHI79cj%2FVNy6wD%2BkLG4vUlAZhTWxwmJFCm6xo3iZ3c1xEiFcGu001RK8kryvINZy47zDTsKs8PZ2zpUYwrquo%2Foi0mYLE4QQRAPoqQb51PFH39MbfAzRZcN7wuOLF2%2FXQp2%2F4D1b%2BRmy%2BHa1WoSL78sVCLLO6kQrWxXcLLk5CMrXj%2FOzAds4kLzal4eQ6tzwyCpdbyR55lQXZeX8i5HfwQ1nUOqt8E3PpikonwXbJ7svv%2BJvf%2Ffr5Yl81afdw888z25plbtrSRk6%2FZDrOcsfyP0dBcFTl2rSpDqztuIz1WTp%2FBFKPoeoF6LBKu5j1MfJZXTVH%2FwGwdRnKRRxrbIzTCjVlXI22mq9lbz1p2ETXAIFK8d%2BCZZO0MW%2BpsdeD5wHBLc1zegjbQ%2Fha%2BXLnN37r59NF1oKegputCmIezuZOIrbE8H

In [26]:
from cobre_ds_utils.adapters.snowflake_adapter import SnowflakeAdapter


user = os.getenv('SNOWFLAKE_USER')
print(user)
account = os.getenv('SNOWFLAKE_ACCOUNT')
warehouse = os.getenv('SNOWFLAKE_WAREHOUSE')
database = os.getenv('SNOWFLAKE_DATABASE')
snowflake_adapter = SnowflakeAdapter(user, account, warehouse, database)

None


AttributeError: 'NoneType' object has no attribute 'find'

In [25]:
from dotenv import load_dotenv
import os

# 1. Cargar variables
load_dotenv('local.env')

# 2. Verificar variables
user = os.getenv('SNOWFLAKE_USER')
account = os.getenv('SNOWFLAKE_ACCOUNT')
warehouse = os.getenv('SNOWFLAKE_WAREHOUSE')
database = os.getenv('SNOWFLAKE_DATABASE')

print(f"Conectando a: {database} en {account} como {user}")

# 3. Crear adapter
if all([user, account, warehouse, database]):
    snowflake_adapter = SnowflakeAdapter(user, account, warehouse, database)
else:
    print("Error: Variables de entorno faltantes")

Conectando a: None en None como None
Error: Variables de entorno faltantes


### Queries

In [390]:
INVOICE_PRICING_QUERY = """
WITH INVOICE AS (
  SELECT
      C.CUSTOMER_ALIASES,
      I.ID
  FROM COBRE_GOLD_DB.SEQUENCE.INVOICES I
  LEFT JOIN COBRE_GOLD_DB.SEQUENCE.CUSTOMERS C
      ON I.CUSTOMER_ID = C.ID
  WHERE 1 = 1
    AND (
     C.CUSTOMER_ALIASES = '{client_id}'
  OR C.CUSTOMER_ALIASES LIKE '{client_id},%'
  OR C.CUSTOMER_ALIASES LIKE '%,{client_id}'
  OR C.CUSTOMER_ALIASES LIKE '%,{client_id},%'
)
    AND BILLING_PERIOD_START :: DATE = '{billing_period_start}'
    AND BILLING_PERIOD_END   :: DATE = '{billing_period_end}'
),
UNION_ALL AS (
  SELECT
       ILI.BILLING_PERIOD_START :: DATE              AS MONTH,
       ILI.INVOICE_ID,
       ILI.TITLE,
       P.PRODUCT_NAME,
       UM.NAME                                  AS USAGE_METRIC,
       BSPP.PRICE_MINIMUM_AMOUNT,
       IFF(BSPP.PRICE_MINIMUM_AMOUNT IS NOT NULL, TRUE, FALSE) AS CONSUMES_SAAS,
       ILI.QUANTITY,
       ILI.RATE,
       UM.PROPERTY_FILTERS,
       BSPP.PRICE_STRUCTURE,
       ILI.CURRENCY,
       ILI.NET_TOTAL,
       ILI.GROSS_TOTAL,
       ILI.CALCULATED_AT
  FROM COBRE_GOLD_DB.SEQUENCE.INVOICE_LINE_ITEMS ILI
  LEFT JOIN COBRE_GOLD_DB.SEQUENCE.PRICES P
      ON ILI.PRICE_ID = P.ID
  LEFT JOIN COBRE_GOLD_DB.SEQUENCE.BILLING_SCHEDULE_PHASE_PRICES BSPP
      ON P.ID = BSPP.PRICE_ID
     AND BSPP.STATUS = 'ACTIVE'
     AND BSPP.PHASE_ARCHIVED_AT IS NULL
  LEFT JOIN COBRE_GOLD_DB.SEQUENCE.USAGE_METRICS UM
      ON UM.ID = BSPP.PRICE_STRUCTURE['usageMetricId'] :: STRING
  INNER JOIN INVOICE I
      ON ILI.INVOICE_ID = I.ID

  QUALIFY
    DENSE_RANK() OVER (
      PARTITION BY ILI.INVOICE_ID
      ORDER BY ILI.CALCULATED_AT DESC
    ) = 1
),
RANKED AS (
  SELECT
    UA.*,
    DENSE_RANK() OVER (
      ORDER BY UA.CALCULATED_AT DESC
    ) AS DATE_NUMING
  FROM UNION_ALL UA
)
SELECT *
FROM RANKED
WHERE DATE_NUMING = 1;
"""




In [379]:
PAYOUTS_MMS_QUERY = """

SELECT AMOUNT,ID                                                               AS MM_ID
     , CLIENT_ID
     , 'COBRE BILLING'                                                  AS EVENTTYPE
     , DATE_TRUNC(
        'DAY',
        TO_DATE(
                CONVERT_TIMEZONE(
                        'UTC',
                        CASE
                            WHEN SOURCE_GEOGRAPHY = 'col' THEN 'America/Bogota'
                            WHEN SOURCE_GEOGRAPHY = 'mex' THEN 'America/Mexico_City'
                            ELSE 'UTC' -- Default timezone if country is unrecognized
                            END,
                        TO_TIMESTAMP_NTZ(CREATED_AT)
                )
        )
       )                                                                AS EVENTTIMESTAMP
     , 'PAYOUT'                                                         AS FLOW
     , UPPER(TYPE)                                                      AS TRANSACTION_TYPE

     -- Categorize origination system
     , CASE
           WHEN COBRE_ORIGIN_CHANNEL = 'api' THEN 'API'
           WHEN COBRE_ORIGIN_CHANNEL NOT IN ('api') THEN 'PORTAL'
           ELSE 'OTHER'
    END                                                                 AS ORIGINATION_SYSTEM

     -- Categorize source account type
     , CASE
           WHEN LOWER(SOURCE_ACCOUNT_PROVIDER_ID) LIKE '%cobre%' THEN 'COBRE BALANCE'
           ELSE 'CONNECT'
    END                                                                 AS SOURCE_ACCOUNT_TYPE

     -- Determine transfer method
     , CASE
           WHEN TYPE = 'payout' THEN
               CASE
                   WHEN DESTINATION_ACCOUNT_INSTITUTION_CODE IS NULL OR SOURCE_ACCOUNT_INSTITUTION_CODE IS NULL
                       THEN 'UNKNOWN'
                   WHEN DESTINATION_ACCOUNT_INSTITUTION_CODE = SOURCE_ACCOUNT_INSTITUTION_CODE THEN 'FAST_PAY'
                   WHEN SOURCE_ACCOUNT_INSTITUTION_CODE = '1023' AND
                        DESTINATION_ACCOUNT_INSTITUTION_CODE IN ('1001', '1002', '1052') THEN 'FAST_PAY'
                   WHEN SOURCE_ACCOUNT_INSTITUTION_CODE = '1007' AND DESTINATION_ACCOUNT_INSTITUTION_CODE = '1507'
                       THEN 'FAST_PAY' -- NEQUI
                   WHEN SOURCE_ACCOUNT_INSTITUTION_CODE = '1051' AND DESTINATION_ACCOUNT_INSTITUTION_CODE = '1551'
                       THEN 'FAST_PAY' -- DAVIPLATA
                   ELSE 'ACH'
                   END
           WHEN TYPE IN ('spei', 'spei_card') THEN 'SPEI'
           WHEN TYPE = 'fast_pay' THEN 'FAST_PAY'
           WHEN TYPE = 'ach' THEN 'ACH'
           ELSE 'UNKNOWN'
    END                                                                 AS TRANSFER_METHOD

     -- Determine destination bank
     , COALESCE(UPPER(DESTINATION_ACCOUNT_INSTITUTION_NAME), 'UNKNOWN') AS DESTINATION_BANK

     -- Determine origin bank
     , COALESCE(
        CASE
            WHEN TYPE = 'payout' THEN UPPER(SOURCE_ACCOUNT_INSTITUTION_NAME)
            WHEN TYPE IN ('fast_pay', 'ach') THEN COALESCE(
                    CASE SOURCE_UNDERLAYING_COBRE_INSTITUTION
                        WHEN '1051' THEN 'BANCO DAVIVIENDA SA'
                        WHEN '1066' THEN 'BANCO COOPERATIVO COOPCENTRAL'
                        WHEN '1023' THEN 'BANCO DE OCCIDENTE'
                        WHEN '1001' THEN 'BANCO DE BOGOTA'
                        WHEN '9999' THEN 'COBRE'
                        WHEN '1019' THEN 'SCOTIABANK COLPATRIA S.A'
                        WHEN '1013' THEN 'BBVA COLOMBIA'
                        WHEN '1507' THEN 'NEQUI'
                        WHEN '1007' THEN 'BANCOLOMBIA'
                        ELSE 'UNKNOWN'
                        END,
                    CASE RIGHT(SUPPLIER, 4)
                        WHEN '1051' THEN 'BANCO DAVIVIENDA SA'
                        WHEN '1066' THEN 'BANCO COOPERATIVO COOPCENTRAL'
                        WHEN '1023' THEN 'BANCO DE OCCIDENTE'
                        WHEN '1001' THEN 'BANCO DE BOGOTA'
                        WHEN '9999' THEN 'COBRE'
                        WHEN '1019' THEN 'SCOTIABANK COLPATRIA S.A'
                        WHEN '1013' THEN 'BBVA COLOMBIA'
                        WHEN '1507' THEN 'NEQUI'
                        WHEN '1007' THEN 'BANCOLOMBIA'
                        ELSE 'COBRE'
                        END,
                    'UNKNOWN'
                                                  )
            WHEN TYPE IN ('spei', 'spei_card') THEN UPPER(SOURCE_ACCOUNT_INSTITUTION_NAME)
            ELSE 'UNKNOWN'
            END, 'UNKNOWN')                                             AS ORIGIN_BANK

     -- Categorize transaction country
     , CASE
           WHEN SOURCE_GEOGRAPHY = 'col' THEN 'COLOMBIA'
           WHEN SOURCE_GEOGRAPHY = 'mex' THEN 'MEXICO'
           ELSE 'OTHER'
    END                                                                 AS COUNTRY

--region Grouping and Aggregation
-- Transaction status
--      , UPPER(STATUS_STATE)                                              AS STATUS

-- Transaction count
--      , COUNT(AMOUNT)                                                    AS TRANSACTIONS

-- Total transaction processed value (TPV)
--      , SUM(AMOUNT)                                                      AS TOTAL_AMOUNT
--      , MAX(
--         TO_DATE(
--                 CONVERT_TIMEZONE(
--                         'UTC',
--                         CASE
--                             WHEN SOURCE_GEOGRAPHY = 'col' THEN 'America/Bogota'
--                             WHEN SOURCE_GEOGRAPHY = 'mex' THEN 'America/Mexico_City'
--                             ELSE 'UTC'
--                             END,
--                         TO_TIMESTAMP_NTZ(UPDATED_AT)
--                 )
--         )
--        )                                                                AS UPDATED_AT
--endregion

FROM COBRE_GOLD_DB.DATAWAREHOUSE.MONEY_MOVEMENTS
WHERE
  -- Exclude specific clients
    CLIENT_ID NOT IN ('cli_pxt_product', 'cli_coe542', 'cli_siikld3ljw', 'PIC00', 'DEM49', 'cli_u2zvkkh50b',
                      'cli_rx0g0ajunr', 'cli_phdanw1wsc', 'cli_vdke2laqnj', 'cli_zkggvlvqf0', 'PXT01', 'PPF594',
                      'SCI528', 'R1.PRL637', 'R1C1E478', 'R1.BBC634', 'PLO429', 'R1.BIL620', 'POG521',
                      'cli_pxzh3oo8cy', 'R1.C1E478', 'cli_8gv2wyvywv', 'CNE614', 'cli_xxz85j60is', 'COS394',
                      'R1.CET436', 'cli_inf001', 'CBC561', 'CNR563', 'CEF564', 'CTS565', 'cli_geyjth0opt', 'CTC602',
                      'cli_1wkwadayff', 'r1.PEL603', 'cli_f3atilxc7k', 'cli_9xyiscji0k', 'cli_l5w6myjnek')

  -- Filter by transaction type and status
  AND TYPE IN ('payout', 'ach', 'fast_pay', 'spei', 'spei_card')
  AND STATUS_STATE IN ('finished', 'completed')
  AND DATE_TRUNC('MONTH',
                 TO_DATE(
                         CONVERT_TIMEZONE(
                                 'UTC',
                                 CASE
                                     WHEN SOURCE_GEOGRAPHY = 'col' THEN 'America/Bogota'
                                     WHEN SOURCE_GEOGRAPHY = 'mex' THEN 'America/Mexico_City'
                                     ELSE 'UTC'
                                     END,
                                 TO_TIMESTAMP_NTZ(CREATED_AT)
                         )
                 )
      ) = '{billing_period_start}'
  AND CLIENT_ID = {client_id}
  --AND CLIENT_ID = 'cli_bdgczxcwd0' -- dac bancolombia
  --AND CLIENT_ID = 'cli_3qjccqlafg'--tiers volume
"""

In [444]:
# Mostrar más columnas y filas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 50)

# Para números decimales
pd.set_option('display.float_format', '{:.2f}'.format)

# Instalar: pip install itables
from itables import show

# queries from sales-ops

### PYOUT

In [427]:
PAYOUTS_MMS_QUERY = """select * from COBRE_GOLD_DB.SALES_OPS.PAYOUT_TRANSACTIONS
    where  UPDATED_AT::DATE between '{billing_period_start}' and '{billing_period_end}'
    and CLIENT_ID = '{client_id}' """


### PYIN

In [497]:
PAYIN_MMS_QUERY = """select * from COBRE_GOLD_DB.SALES_OPS.PAYIN_TRANSACTIONS
    where  UPDATED_AT::DATE between '{billing_period_start}' and '{billing_period_end}'
    and CLIENT_ID = '{client_id}' """

### balance rechargue

In [489]:
BALANCE_RECHARGES_MMS_QUERY = """select * from COBRE_GOLD_DB.SALES_OPS.BALANCE_RECHARGES
    where  UPDATED_AT::DATE between '{billing_period_start}' and '{billing_period_end}'
    and CLIENT_ID = '{client_id}' """

### daC transactions

In [485]:
DAC_MMS_QUERY = """select * from COBRE_GOLD_DB.SALES_OPS.DAC_TRANSACTIONS
    where  UPDATED_AT::DATE between '{billing_period_start}' and '{billing_period_end}'
    and CLIENT_ID = '{client_id}' """

### Processing

In [ ]:
# invoice_pricing_df = snowflake_adapter.execute_query(
#     INVOICE_PRICING_QUERY.format(billing_period_start='2025-05-01', client_id=client_id)
# )

In [514]:
INICIAL='2025-05-01'
FINAL='2025-05-31'
client_id='cli_3qjccqlafg'

GET INVOICE STRUCTURE FOR EVERY CLIENT

In [515]:
invoice_pricing_df = snowflake_adapter.query_df(
    INVOICE_PRICING_QUERY.format(billing_period_start=INICIAL, billing_period_end=FINAL, client_id=client_id)  
)
invoice_pricing_df['PROPERTY_FILTERS'] = invoice_pricing_df['PROPERTY_FILTERS'].apply(
    lambda x: json.loads(x) if pd.notna(x) and x is not None else None
)
invoice_pricing_df['PRICE_STRUCTURE'] = invoice_pricing_df['PRICE_STRUCTURE'].apply(
    lambda x: json.loads(x) if pd.notna(x) and x is not None else None
)

show(invoice_pricing_df.head(15), maxBytes=0)

Loading ITables v2.4.0 from the internet... (need help?)


### ALL TABLES CALCULATED

In [517]:
payouts_mms_df = snowflake_adapter.query_df(
    PAYOUTS_MMS_QUERY.format(billing_period_start='2025-05-01', billing_period_end='2025-05-31', client_id=client_id)
)
show(payouts_mms_df.head(150), maxBytes=0)


Loading ITables v2.4.0 from the internet... (need help?)


In [511]:
payin_mms_df = snowflake_adapter.query_df(
    PAYIN_MMS_QUERY.format(billing_period_start='2025-05-01', billing_period_end='2025-05-31', client_id=client_id)
)
show(payin_mms_df.head(150), maxBytes=0)

Loading ITables v2.4.0 from the internet... (need help?)


In [512]:
dac_mms_df = snowflake_adapter.query_df(
    DAC_MMS_QUERY.format(billing_period_start='2025-05-01', billing_period_end='2025-05-31', client_id=client_id)
)
show(dac_mms_df.head(150), maxBytes=0)

Loading ITables v2.4.0 from the internet... (need help?)


In [513]:
balance_recharges_mms_df = snowflake_adapter.query_df(
    BALANCE_RECHARGES_MMS_QUERY.format(billing_period_start='2025-05-01', billing_period_end='2025-05-31', client_id=client_id)
)
show(balance_recharges_mms_df.head(150), maxBytes=0)


Loading ITables v2.4.0 from the internet... (need help?)


# APPLING MATCH RULES AND  PRICE RULES

v25

In [537]:
import pandas as pd
import ast
from typing import Union, List

def apply_rules_to_mm(mm: pd.DataFrame, rules: pd.DataFrame, debug: bool = False) -> pd.DataFrame:
    """
    Aplica las reglas de filtrado del dataframe rules al dataframe mm.

    Args:
        mm (pd.DataFrame): DataFrame con los datos de transacciones
        rules (pd.DataFrame): DataFrame con las reglas que contiene PROPERTY_FILTERS,
                              PRODUCT_NAME, PRICE_STRUCTURE, PRICE_MINIMUM_AMOUNT, CONSUMES_SAAS
        debug (bool): Si True, muestra información detallada del procesamiento

    Returns:
        pd.DataFrame: DataFrame mm con cuatro columnas adicionales:
                     'match'               -> PRODUCT_NAME que hicieron match
                     'prices_rules'        -> PRICE_STRUCTURE correspondiente
                     'price_minimum_amount'-> PRICE_MINIMUM_AMOUNT de la regla
                     'consumes_saas'       -> CONSUMES_SAAS de la regla
                     (listas si hay múltiples matches)
    """
    # Crear copia para los resultados
    mm_result = mm.copy()
    # Inicializar las nuevas columnas
    for col in ['match', 'prices_rules', 'price_minimum_amount', 'consumes_saas']:
        mm_result[col] = None
        mm_result[col] = mm_result[col].astype('object')

    # Para matching case-insensitive
    mm_lower = mm.copy()
    mm_lower.columns = mm_lower.columns.str.lower()

    # Iterar cada fila de mm
    for mm_idx, mm_row in mm_lower.iterrows():
        matches = []
        price_structures = []
        min_amounts = []
        saas_flags = []

        if debug and mm_idx < 3:
            print(f"\n=== PROCESANDO FILA MM {mm_idx} ===")

        # Iterar cada regla
        for rules_idx, rules_row in rules.iterrows():
            try:
                raw = rules_row.get('PROPERTY_FILTERS', None)
                if pd.isna(raw):
                    continue

                # Parsear filtro
                property_filter = raw if isinstance(raw, dict) else ast.literal_eval(str(raw))

                match_found = True
                # Verificar cada condición
                for key, vals in property_filter.items():
                    key_l = key.lower()
                    if key_l == 'validation':
                        continue
                    if key_l not in mm_lower.columns:
                        match_found = False
                        break
                    val = str(mm_lower.at[mm_idx, key_l]).lower().strip()
                    allowed = [str(v).lower().strip() for v in vals]
                    if val not in allowed:
                        match_found = False
                        break

                if match_found:
                    # Guardar info de la regla
                    matches.append(rules_row.get('PRODUCT_NAME'))
                    price_structures.append(rules_row.get('PRICE_STRUCTURE'))
                    min_amounts.append(rules_row.get('PRICE_MINIMUM_AMOUNT'))
                    saas_flags.append(rules_row.get('CONSUMES_SAAS'))

            except Exception as e:
                if debug:
                    print(f"Error en regla {rules_idx}: {e}")
                continue

        # Asignar resultados según cantidad de matches
        if len(matches) == 1:
            mm_result.at[mm_idx, 'match'] = matches[0]
            mm_result.at[mm_idx, 'prices_rules'] = price_structures[0]
            mm_result.at[mm_idx, 'price_minimum_amount'] = min_amounts[0]
            mm_result.at[mm_idx, 'consumes_saas'] = saas_flags[0]
        elif len(matches) > 1:
            mm_result.at[mm_idx, 'match'] = matches
            mm_result.at[mm_idx, 'prices_rules'] = price_structures
            mm_result.at[mm_idx, 'price_minimum_amount'] = min_amounts
            mm_result.at[mm_idx, 'consumes_saas'] = saas_flags

        if debug and mm_idx < 3:
            print(f"  Matches: {matches}")
            print(f"  Price rules: {price_structures}")
            print(f"  Min amounts: {min_amounts}")
            print(f"  Consumes SaaS: {saas_flags}")

    return mm_result


def calculate_linear_pricing(price_dict: dict, transaction_amount: float, match_name: str, rule_number: int) -> dict:
    """
    Calcula el revenue para pricing tipo LINEAR
    """
    is_percentage = price_dict.get('isPricePercentage', False)
    price_per_unit = float(price_dict.get('pricePerUnit', 0))
    
    if is_percentage:
        # Calcular porcentaje del monto
        percentage_rate = price_per_unit / 100
        revenue = transaction_amount * percentage_rate
        calculation_method = "Porcentaje"
        calculation_formula = f"{transaction_amount:,} × {price_per_unit}% = {revenue:,.2f}"
    else:
        # Carga fija
        revenue = price_per_unit
        calculation_method = "Carga fija"
        calculation_formula = f"{price_per_unit:,.2f}"
    
    return {
        'regla_numero': rule_number,
        'match_name': match_name,
        'pricing_type': 'LINEAR',
        'is_percentage': is_percentage,
        'price_per_unit': price_per_unit,
        'transaction_amount': transaction_amount,
        'revenue': revenue,
        'calculation_method': calculation_method,
        'calculation_formula': calculation_formula
    }


def calculate_volume_pricing(price_dict: dict, transaction_amount: float, match_name: str, rule_number: int, transaction_count: int = 1) -> dict:
    """
    Calcula el revenue para pricing tipo VOLUME basado en tiers
    
    Args:
        price_dict: Diccionario con la estructura de pricing VOLUME
        transaction_amount: Monto de la transacción
        match_name: Nombre del match
        rule_number: Número de regla
        transaction_count: Cantidad de transacciones (por defecto 1)
    """
    tiers = price_dict.get('tiers', [])
    if not tiers:
        return {
            'regla_numero': rule_number,
            'match_name': match_name,
            'pricing_type': 'VOLUME',
            'revenue': 0,
            'calculation_method': 'Error',
            'calculation_formula': 'No hay tiers definidos'
        }
    
    # Encontrar el tier apropiado basado en transaction_count
    selected_tier = None
    for tier in tiers:
        upper_bound = tier.get('upperBound')
        if upper_bound is None:
            # Último tier sin límite superior
            selected_tier = tier
            break
        elif transaction_count <= int(upper_bound):
            selected_tier = tier
            break
    
    if selected_tier is None:
        selected_tier = tiers[-1]  # Usar el último tier si no se encuentra match
    
    # Extraer valores del tier seleccionado
    is_percentage = selected_tier.get('isPricePercentage', False)
    price = float(selected_tier.get('price', 0))
    fee = float(selected_tier.get('fee', 0))
    upper_bound = selected_tier.get('upperBound', 'Sin límite')
    
    # Calcular revenue según isPricePercentage
    if is_percentage:
        percentage_rate = price / 100
        base_revenue = transaction_amount * percentage_rate
        calculation_method = "Volumen - Porcentaje"
        base_formula = f"{transaction_amount:,} × {price}%"
    else:
        base_revenue = price
        calculation_method = "Volumen - Carga fija"
        base_formula = f"{price:,.2f}"
    
    # Agregar fee si existe
    total_revenue = base_revenue + fee
    
    # Crear fórmula completa
    if fee > 0:
        calculation_formula = f"{base_formula} + fee({fee:,.2f}) = {total_revenue:,.2f}"
    else:
        calculation_formula = f"{base_formula} = {total_revenue:,.2f}"
    
    tier_info = f"Tier: hasta {upper_bound} transacciones"
    
    return {
        'regla_numero': rule_number,
        'match_name': match_name,
        'pricing_type': 'VOLUME',
        'tier_selected': tier_info,
        'is_percentage': is_percentage,
        'price': price,
        'fee': fee,
        'transaction_amount': transaction_amount,
        'transaction_count': transaction_count,
        'revenue': total_revenue,
        'calculation_method': calculation_method,
        'calculation_formula': calculation_formula
    }


def calculate_transaction_counts_by_match(df_with_matches: pd.DataFrame) -> dict:
    """
    Calcula la cantidad de transacciones por cada tipo de match.
    Maneja tanto matches únicos como múltiples matches por fila.
    
    Args:
        df_with_matches: DataFrame con columnas 'match' y 'prices_rules'
    
    Returns:
        dict: Mapping de match_name -> transaction_count
    """
    match_counts = {}
    
    for idx, row in df_with_matches.iterrows():
        match_value = row['match']
        
        # Si no hay match, skip
        if pd.isna(match_value) or match_value is None:
            continue
        
        # Manejar múltiples matches
        if isinstance(match_value, list):
            match_names = match_value
        else:
            match_names = [match_value]
        
        # Contar cada match
        for match_name in match_names:
            if match_name not in match_counts:
                match_counts[match_name] = 0
            match_counts[match_name] += 1
    
    return match_counts


def calculate_revenue(df_with_matches: pd.DataFrame, amount_column: str = 'TOTAL_AMOUNT', debug: bool = False) -> pd.DataFrame:
    """
    Calcula el revenue por transacción basado en la columna 'prices_rules'.
    Soporta pricingType: 'LINEAR' y 'VOLUME'
    Para VOLUME, calcula automáticamente la cantidad de transacciones por cada match específico.
    
    Args:
        df_with_matches (pd.DataFrame): DataFrame que ya tiene las columnas 'match' y 'prices_rules'
        amount_column (str): Nombre de la columna que contiene el monto de la transacción
        debug (bool): Si True, muestra información detallada del cálculo
    
    Returns:
        pd.DataFrame: DataFrame con columnas adicionales 'revenue' y 'revenue_calculation_details'
    """
    # Crear una copia del dataframe
    result_df = df_with_matches.copy()
    result_df['revenue'] = 0.0
    result_df['revenue_calculation_details'] = None
    result_df['revenue_calculation_details'] = result_df['revenue_calculation_details'].astype('object')
    
    # Verificar si existe la columna de amount
    if amount_column not in result_df.columns:
        print(f"Advertencia: Columna '{amount_column}' no encontrada. Usando valor 0 para cálculos de porcentaje.")
        result_df[amount_column] = 0.0
    
    # PASO 1: Calcular automáticamente las cantidades de transacciones por match
    transaction_counts_by_match = calculate_transaction_counts_by_match(df_with_matches)
    
    if debug:
        print("🔢 CONTEO AUTOMÁTICO DE TRANSACCIONES POR MATCH:")
        for match_name, count in transaction_counts_by_match.items():
            print(f"   {match_name}: {count} transacciones")
        print()
    
    # PASO 2: Calcular revenue para cada fila
    for idx, row in result_df.iterrows():
        total_revenue = 0.0
        calculation_details = []
        
        # Si no hay prices_rules, el revenue es 0
        prices_rules_value = row['prices_rules']
        if prices_rules_value is None or (isinstance(prices_rules_value, float) and pd.isna(prices_rules_value)):
            if debug and idx < 5:
                print(f"Fila {idx}: Sin reglas de pricing")
            continue
        
        # Obtener el monto de la transacción
        transaction_amount = float(row.get(amount_column, 0))
        
        if debug and idx < 5:
            print(f"\n=== CALCULANDO REVENUE PARA FILA {idx} ===")
            print(f"Amount: {transaction_amount:,}")
            print(f"Match: {row['match']}")
        
        # Determinar si hay múltiples reglas o una sola
        if isinstance(prices_rules_value, list):
            price_structures = prices_rules_value
            match_names = row['match'] if isinstance(row['match'], list) else [row['match']]
            if debug and idx < 5:
                print(f"📋 MÚLTIPLES REGLAS DETECTADAS: {len(price_structures)} reglas")
        else:
            price_structures = [prices_rules_value]
            match_names = [row['match']]
            if debug and idx < 5:
                print(f"📋 UNA SOLA REGLA DETECTADA")
        
        # Calcular revenue para cada price structure
        for i, price_structure in enumerate(price_structures):
            try:
                # Saltar si es None o vacío
                if price_structure is None or price_structure == {}:
                    if debug and idx < 5:
                        print(f"  Regla {i+1}: SALTADA (vacía)")
                    continue
                
                # Convertir a dict si es string
                if isinstance(price_structure, str):
                    price_dict = ast.literal_eval(price_structure)
                else:
                    price_dict = price_structure
                
                # Obtener tipo de pricing
                pricing_type = price_dict.get('pricingType', '').upper()
                match_name = match_names[i] if i < len(match_names) else f"Match_{i}"
                
                # Calcular según el tipo de pricing
                if pricing_type == 'LINEAR':
                    detail = calculate_linear_pricing(price_dict, transaction_amount, match_name, i + 1)
                    
                elif pricing_type == 'VOLUME':
                    # OBTENER AUTOMÁTICAMENTE LA CANTIDAD DE TRANSACCIONES PARA ESTE MATCH ESPECÍFICO
                    transaction_count = transaction_counts_by_match.get(match_name, 1)
                    
                    if debug and idx < 5:
                        print(f"  📊 Usando transaction_count = {transaction_count} para match '{match_name}'")
                    
                    detail = calculate_volume_pricing(price_dict, transaction_amount, match_name, i + 1, transaction_count)
                    
                else:
                    if debug and idx < 5:
                        print(f"  Regla {i+1}: SALTADA (pricingType '{pricing_type}' no soportado)")
                    continue
                
                # Agregar revenue al total
                revenue_for_this_rule = detail['revenue']
                total_revenue += revenue_for_this_rule
                calculation_details.append(detail)
                
                if debug and idx < 5:
                    print(f"  ✅ Regla {i+1} ({match_name}) - {pricing_type}:")
                    print(f"      Método: {detail['calculation_method']}")
                    print(f"      Cálculo: {detail['calculation_formula']}")
                    print(f"      Revenue: {revenue_for_this_rule:,.2f}")
                    if pricing_type == 'VOLUME':
                        print(f"      Tier: {detail.get('tier_selected', 'N/A')}")
                        print(f"      Transaction Count usado: {detail.get('transaction_count', 'N/A')}")
                
            except Exception as e:
                if debug and idx < 5:
                    print(f"  ❌ Error en regla {i+1}: {e}")
                continue
        
        # Asignar resultados
        result_df.at[idx, 'revenue'] = total_revenue
        if calculation_details:
            result_df.at[idx, 'revenue_calculation_details'] = calculation_details
        
        if debug and idx < 5:
            print(f"💰 TOTAL REVENUE: {total_revenue:,.2f}")
            if len(calculation_details) > 1:
                print(f"    (Suma de {len(calculation_details)} reglas)")
    
    return result_df


def test_multiple_rules_scenario():
    """
    Función de prueba que demuestra múltiples reglas por transacción
    """
    # Crear datos de prueba con casos específicos
    test_data = {
        'MM_ID': ['tx_no_rules', 'tx_single_percentage', 'tx_single_fixed', 'tx_multiple_rules', 'tx_mixed_types'],
        'TOTAL_AMOUNT': [100000, 50000, 200000, 150000, 80000],
        'match': [
            None,  # Sin match
            'Cash-in Percentage Fee',  # Una sola regla de porcentaje
            'Any Cash-out Transactions',  # Una sola regla fija
            ['DaC Transactions', 'Cash-in Transactions', 'Additional Fee'],  # Múltiples reglas
            ['Platform Fee', 'Cash-in Percentage']  # Reglas mixtas (una no LINEAR)
        ],
        'prices_rules': [
            None,  # Sin reglas
            
            # Una regla de porcentaje: 0.20% de 50,000 = 100
            {'isPricePercentage': True, 'pricePerUnit': '0.20', 'pricingType': 'LINEAR'},
            
            # Una regla fija: 1,600
            {'isPricePercentage': False, 'pricePerUnit': '1600.00', 'pricingType': 'LINEAR'},
            
            # Múltiples reglas LINEAR: 1500 + (150,000 * 0.15%) + 800 = 1500 + 225 + 800 = 2525
            [
                {'isPricePercentage': False, 'pricePerUnit': '1500.00', 'pricingType': 'LINEAR'},
                {'isPricePercentage': True, 'pricePerUnit': '0.15', 'pricingType': 'LINEAR'}, 
                {'isPricePercentage': False, 'pricePerUnit': '800.00', 'pricingType': 'LINEAR'}
            ],
            
            # Reglas mixtas (una no LINEAR se salta): solo 80,000 * 0.10% = 80
            [
                {'price': '100000.00', 'pricingType': 'FIXED'},  # Se salta
                {'isPricePercentage': True, 'pricePerUnit': '0.10', 'pricingType': 'LINEAR'}
            ]
        ]
    }
    
    test_df = pd.DataFrame(test_data)
    
    print("=== DATOS DE PRUEBA - MÚLTIPLES REGLAS ===")
    for idx, row in test_df.iterrows():
        print(f"\n{row['MM_ID']}:")
        print(f"  Amount: {row['TOTAL_AMOUNT']:,}")
        print(f"  Match: {row['match']}")
        print(f"  Rules: {row['prices_rules']}")
    
    print("\n" + "="*60)
    print("CALCULANDO REVENUE CON DEBUG")
    print("="*60)
    
    result = calculate_revenue(test_df, debug=True)
    
    print("\n" + "="*60)
    print("RESUMEN DE RESULTADOS")
    print("="*60)
    
    for idx, row in result.iterrows():
        print(f"\n🏷️  {row['MM_ID']}:")
        print(f"   💰 Amount: {row['TOTAL_AMOUNT']:,}")
        print(f"   📊 Revenue: {row['revenue']:,.2f}")
        
        if row['revenue_calculation_details']:
            print(f"   📋 Detalles del cálculo:")
            for detail in row['revenue_calculation_details']:
                print(f"      Regla {detail['regla_numero']}: {detail['match_name']}")
                print(f"        └─ {detail['calculation_method']}: {detail['calculation_formula']}")
        else:
            print(f"   📋 Sin reglas aplicables")
    
    return result


def test_volume_pricing_scenario():
    """
    Función de prueba específica para demostrar pricing tipo VOLUME
    """
    # Crear datos de prueba con casos de VOLUME pricing
    test_data = {
        'MM_ID': ['vol_low', 'vol_mid', 'vol_high', 'vol_percentage', 'vol_with_fee'],
        'TOTAL_AMOUNT': [100000, 200000, 500000, 150000, 300000],
        'match': [
            'Volume Pricing Basic',
            'Volume Pricing Basic', 
            'Volume Pricing Basic',
            'Volume Pricing Percentage',
            'Volume Pricing With Fee'
        ],
        'prices_rules': [
            # Tier 1: hasta 100 transacciones - precio 2000
            {'pricingType': 'VOLUME', 'tiers': [
                {'fee': '0.00', 'isPricePercentage': False, 'price': '2000.00', 'upperBound': '100'},
                {'fee': '0.00', 'isPricePercentage': False, 'price': '1900.00', 'upperBound': '500'},
                {'fee': '0.00', 'isPricePercentage': False, 'price': '1700.00'}
            ]},
            
            # Tier 2: hasta 500 transacciones - precio 1900
            {'pricingType': 'VOLUME', 'tiers': [
                {'fee': '0.00', 'isPricePercentage': False, 'price': '2000.00', 'upperBound': '100'},
                {'fee': '0.00', 'isPricePercentage': False, 'price': '1900.00', 'upperBound': '500'},
                {'fee': '0.00', 'isPricePercentage': False, 'price': '1700.00'}
            ]},
            
            # Tier 3: más de 500 transacciones - precio 1700
            {'pricingType': 'VOLUME', 'tiers': [
                {'fee': '0.00', 'isPricePercentage': False, 'price': '2000.00', 'upperBound': '100'},
                {'fee': '0.00', 'isPricePercentage': False, 'price': '1900.00', 'upperBound': '500'},
                {'fee': '0.00', 'isPricePercentage': False, 'price': '1700.00'}
            ]},
            
            # Pricing por porcentaje en tiers
            {'pricingType': 'VOLUME', 'tiers': [
                {'fee': '0.00', 'isPricePercentage': True, 'price': '0.50', 'upperBound': '100'},
                {'fee': '0.00', 'isPricePercentage': True, 'price': '0.40', 'upperBound': '500'},
                {'fee': '0.00', 'isPricePercentage': True, 'price': '0.30'}
            ]},
            
            # Pricing con fee adicional
            {'pricingType': 'VOLUME', 'tiers': [
                {'fee': '500.00', 'isPricePercentage': False, 'price': '1500.00', 'upperBound': '200'},
                {'fee': '300.00', 'isPricePercentage': False, 'price': '1300.00', 'upperBound': '500'},
                {'fee': '100.00', 'isPricePercentage': False, 'price': '1100.00'}
            ]}
        ]
    }
    
    test_df = pd.DataFrame(test_data)
    
    print("=== DATOS DE PRUEBA - VOLUME PRICING ===")
    for idx, row in test_df.iterrows():
        print(f"\n{row['MM_ID']}:")
        print(f"  Amount: {row['TOTAL_AMOUNT']:,}")
        print(f"  Match: {row['match']}")
        
        # Mostrar estructura de tiers
        price_rule = row['prices_rules']
        if isinstance(price_rule, dict) and 'tiers' in price_rule:
            print(f"  Tiers disponibles:")
            for i, tier in enumerate(price_rule['tiers']):
                upper_bound = tier.get('upperBound', 'Sin límite')
                price = tier.get('price', 0)
                is_perc = tier.get('isPricePercentage', False)
                fee = tier.get('fee', 0)
                price_type = "%" if is_perc else "fijo"
                fee_text = f" + fee({fee})" if float(fee) > 0 else ""
                print(f"    Tier {i+1}: hasta {upper_bound} tx → {price} ({price_type}){fee_text}")
    
    print("\n" + "="*70)
    print("CALCULANDO REVENUE VOLUME PRICING CON DEBUG")
    print("="*70)
    
    result = calculate_revenue(test_df, debug=True)
    
    print("\n" + "="*60)
    print("RESUMEN DE RESULTADOS")
    print("="*60)
    
    for idx, row in result.iterrows():
        print(f"\n🏷️  {row['MM_ID']}:")
        print(f"   💰 Amount: {row['TOTAL_AMOUNT']:,}")
        print(f"   📊 Revenue: {row['revenue']:,.2f}")
        
        if row['revenue_calculation_details']:
            print(f"   📋 Detalles del cálculo:")
            for detail in row['revenue_calculation_details']:
                print(f"      Regla {detail['regla_numero']}: {detail['match_name']}")
                print(f"        └─ {detail['calculation_method']}: {detail['calculation_formula']}")
                if detail['pricing_type'] == 'VOLUME':
                    print(f"        └─ {detail.get('tier_selected', 'N/A')}")
        else:
            print(f"   📋 Sin reglas aplicables")
    
    return result


def test_mixed_pricing_scenario():
    """
    Función que demuestra cómo funciona con reglas mixtas: LINEAR y VOLUME en el mismo dataset
    """
    test_data = {
        'MM_ID': ['mixed_1', 'mixed_2', 'mixed_3', 'mixed_4'],
        'TOTAL_AMOUNT': [100000, 200000, 50000, 300000],
        'match': [
            ['Linear Fee', 'Volume Fee'],  # Múltiples reglas: una LINEAR y una VOLUME
            'Volume Only',  # Solo VOLUME
            'Linear Only',  # Solo LINEAR
            ['Volume Tier 1', 'Volume Tier 2']  # Múltiples VOLUME
        ],
        'prices_rules': [
            # Múltiples reglas: LINEAR + VOLUME
            [
                {'isPricePercentage': True, 'pricePerUnit': '0.25', 'pricingType': 'LINEAR'},
                {'pricingType': 'VOLUME', 'tiers': [
                    {'fee': '0.00', 'isPricePercentage': False, 'price': '1000.00', 'upperBound': '100'},
                    {'fee': '0.00', 'isPricePercentage': False, 'price': '800.00'}
                ]}
            ],
            
            # Solo VOLUME
            {'pricingType': 'VOLUME', 'tiers': [
                {'fee': '100.00', 'isPricePercentage': False, 'price': '2000.00', 'upperBound': '200'},
                {'fee': '50.00', 'isPricePercentage': False, 'price': '1800.00'}
            ]},
            
            # Solo LINEAR
            {'isPricePercentage': False, 'pricePerUnit': '1500.00', 'pricingType': 'LINEAR'},
            
            # Múltiples VOLUME
            [
                {'pricingType': 'VOLUME', 'tiers': [
                    {'fee': '0.00', 'isPricePercentage': True, 'price': '0.30', 'upperBound': '500'},
                    {'fee': '0.00', 'isPricePercentage': True, 'price': '0.25'}
                ]},
                {'pricingType': 'VOLUME', 'tiers': [
                    {'fee': '200.00', 'isPricePercentage': False, 'price': '500.00', 'upperBound': '1000'},
                    {'fee': '100.00', 'isPricePercentage': False, 'price': '400.00'}
                ]}
            ]
        ]
    }
    
    test_df = pd.DataFrame(test_data)
    
    print("=== DATOS DE PRUEBA - PRICING MIXTO (LINEAR + VOLUME) ===")
    for idx, row in test_df.iterrows():
        print(f"\n{row['MM_ID']}:")
        print(f"  Amount: {row['TOTAL_AMOUNT']:,}")
        print(f"  Match: {row['match']}")
        
        prices_rules = row['prices_rules']
        if isinstance(prices_rules, list):
            print(f"  Reglas múltiples ({len(prices_rules)}):")
            for i, rule in enumerate(prices_rules):
                pricing_type = rule.get('pricingType', 'UNKNOWN')
                print(f"    Regla {i+1}: {pricing_type}")
        else:
            pricing_type = prices_rules.get('pricingType', 'UNKNOWN')
            print(f"  Regla única: {pricing_type}")
    
    print("\n" + "="*70)
    print("CALCULANDO REVENUE CON PRICING MIXTO Y CONTEO AUTOMÁTICO")
    print("="*70)
    
    # Usar la función calculate_revenue que ahora cuenta automáticamente
    result = calculate_revenue(test_df, amount_column='TOTAL_AMOUNT', debug=True)
    
    print("\n" + "="*70)
    print("RESUMEN DE RESULTADOS")
    print("="*70)
    
    for idx, row in result.iterrows():
        print(f"\n🏷️  {row['MM_ID']}:")
        print(f"   💰 Amount: {row['TOTAL_AMOUNT']:,}")
        print(f"   📊 Revenue: {row['revenue']:,.2f}")
        
        if row['revenue_calculation_details']:
            print(f"   📋 Detalles del cálculo:")
            for detail in row['revenue_calculation_details']:
                print(f"      Regla {detail['regla_numero']}: {detail['match_name']} ({detail['pricing_type']})")
                print(f"        └─ {detail['calculation_method']}: {detail['calculation_formula']}")
                if detail['pricing_type'] == 'VOLUME':
                    print(f"        └─ {detail.get('tier_selected', 'N/A')}")
        else:
            print(f"   📋 Sin reglas aplicables")
    
    return result


def test_automatic_transaction_counting():
    """
    Demuestra cómo funciona el conteo automático de transacciones por match para VOLUME pricing
    """
    # Crear datos de prueba que simulan un dataset real con múltiples transacciones
    # donde diferentes transacciones hacen match con diferentes reglas
    test_data = {
        'MM_ID': [
            'tx_001', 'tx_002', 'tx_003', 'tx_004', 'tx_005',  # Cash-out (5 transacciones)
            'tx_006', 'tx_007', 'tx_008',                       # Cash-in (3 transacciones)  
            'tx_009', 'tx_010', 'tx_011', 'tx_012',             # DaC Bancolombia (4 transacciones)
            'tx_013', 'tx_014'                                  # Multiple matches (2 transacciones)
        ],
        'TOTAL_AMOUNT': [100000, 150000, 200000, 180000, 120000,   # Cash-out amounts
                        80000, 90000, 75000,                        # Cash-in amounts
                        60000, 70000, 65000, 85000,                 # DaC amounts
                        110000, 130000],                            # Multiple match amounts
        'match': [
            # 5 transacciones con "Any Cash-out Transactions"
            'Any Cash-out Transactions', 'Any Cash-out Transactions', 'Any Cash-out Transactions',
            'Any Cash-out Transactions', 'Any Cash-out Transactions',
            
            # 3 transacciones con "Cash-in Transactions"
            'Cash-in Transactions', 'Cash-in Transactions', 'Cash-in Transactions',
            
            # 4 transacciones con "DaC Transactions (Bancolombia)"
            'DaC Transactions (Bancolombia)', 'DaC Transactions (Bancolombia)',
            'DaC Transactions (Bancolombia)', 'DaC Transactions (Bancolombia)',
            
            # 2 transacciones con múltiples matches
            ['Any Cash-out Transactions', 'Volume Bonus'],
            ['Any Cash-out Transactions', 'Volume Bonus']
        ],
        'prices_rules': [
            # Cash-out con VOLUME pricing (5 transacciones → tier medio)
            {'pricingType': 'VOLUME', 'tiers': [
                {'fee': '0.00', 'isPricePercentage': False, 'price': '2000.00', 'upperBound': '3'},    # 1-3 tx
                {'fee': '100.00', 'isPricePercentage': False, 'price': '1800.00', 'upperBound': '10'}, # 4-10 tx
                {'fee': '200.00', 'isPricePercentage': False, 'price': '1600.00'}                      # 10+ tx
            ]},
            {'pricingType': 'VOLUME', 'tiers': [
                {'fee': '0.00', 'isPricePercentage': False, 'price': '2000.00', 'upperBound': '3'},
                {'fee': '100.00', 'isPricePercentage': False, 'price': '1800.00', 'upperBound': '10'},
                {'fee': '200.00', 'isPricePercentage': False, 'price': '1600.00'}
            ]},
            {'pricingType': 'VOLUME', 'tiers': [
                {'fee': '0.00', 'isPricePercentage': False, 'price': '2000.00', 'upperBound': '3'},
                {'fee': '100.00', 'isPricePercentage': False, 'price': '1800.00', 'upperBound': '10'},
                {'fee': '200.00', 'isPricePercentage': False, 'price': '1600.00'}
            ]},
            {'pricingType': 'VOLUME', 'tiers': [
                {'fee': '0.00', 'isPricePercentage': False, 'price': '2000.00', 'upperBound': '3'},
                {'fee': '100.00', 'isPricePercentage': False, 'price': '1800.00', 'upperBound': '10'},
                {'fee': '200.00', 'isPricePercentage': False, 'price': '1600.00'}
            ]},
            {'pricingType': 'VOLUME', 'tiers': [
                {'fee': '0.00', 'isPricePercentage': False, 'price': '2000.00', 'upperBound': '3'},
                {'fee': '100.00', 'isPricePercentage': False, 'price': '1800.00', 'upperBound': '10'},
                {'fee': '200.00', 'isPricePercentage': False, 'price': '1600.00'}
            ]},
            
            # Cash-in con VOLUME pricing (3 transacciones → tier bajo)
            {'pricingType': 'VOLUME', 'tiers': [
                {'fee': '50.00', 'isPricePercentage': True, 'price': '1.00', 'upperBound': '5'},        # 1-5 tx
                {'fee': '25.00', 'isPricePercentage': True, 'price': '0.80', 'upperBound': '15'},       # 6-15 tx
                {'fee': '0.00', 'isPricePercentage': True, 'price': '0.60'}                            # 15+ tx
            ]},
            {'pricingType': 'VOLUME', 'tiers': [
                {'fee': '50.00', 'isPricePercentage': True, 'price': '1.00', 'upperBound': '5'},
                {'fee': '25.00', 'isPricePercentage': True, 'price': '0.80', 'upperBound': '15'},
                {'fee': '0.00', 'isPricePercentage': True, 'price': '0.60'}
            ]},
            {'pricingType': 'VOLUME', 'tiers': [
                {'fee': '50.00', 'isPricePercentage': True, 'price': '1.00', 'upperBound': '5'},
                {'fee': '25.00', 'isPricePercentage': True, 'price': '0.80', 'upperBound': '15'},
                {'fee': '0.00', 'isPricePercentage': True, 'price': '0.60'}
            ]},
            
            # DaC con VOLUME pricing (4 transacciones → tier bajo)  
            {'pricingType': 'VOLUME', 'tiers': [
                {'fee': '0.00', 'isPricePercentage': False, 'price': '1500.00', 'upperBound': '10'},    # 1-10 tx
                {'fee': '0.00', 'isPricePercentage': False, 'price': '1200.00', 'upperBound': '25'},    # 11-25 tx
                {'fee': '0.00', 'isPricePercentage': False, 'price': '1000.00'}                        # 25+ tx
            ]},
            {'pricingType': 'VOLUME', 'tiers': [
                {'fee': '0.00', 'isPricePercentage': False, 'price': '1500.00', 'upperBound': '10'},
                {'fee': '0.00', 'isPricePercentage': False, 'price': '1200.00', 'upperBound': '25'},
                {'fee': '0.00', 'isPricePercentage': False, 'price': '1000.00'}
            ]},
            {'pricingType': 'VOLUME', 'tiers': [
                {'fee': '0.00', 'isPricePercentage': False, 'price': '1500.00', 'upperBound': '10'},
                {'fee': '0.00', 'isPricePercentage': False, 'price': '1200.00', 'upperBound': '25'},
                {'fee': '0.00', 'isPricePercentage': False, 'price': '1000.00'}
            ]},
            {'pricingType': 'VOLUME', 'tiers': [
                {'fee': '0.00', 'isPricePercentage': False, 'price': '1500.00', 'upperBound': '10'},
                {'fee': '0.00', 'isPricePercentage': False, 'price': '1200.00', 'upperBound': '25'},
                {'fee': '0.00', 'isPricePercentage': False, 'price': '1000.00'}
            ]},
            
            # Múltiples matches (cada uno cuenta por separado)
            [
                {'pricingType': 'VOLUME', 'tiers': [  # "Any Cash-out Transactions" = 7 total (5+2)
                    {'fee': '0.00', 'isPricePercentage': False, 'price': '2000.00', 'upperBound': '3'},
                    {'fee': '100.00', 'isPricePercentage': False, 'price': '1800.00', 'upperBound': '10'},
                    {'fee': '200.00', 'isPricePercentage': False, 'price': '1600.00'}
                ]},
                {'pricingType': 'VOLUME', 'tiers': [  # "Volume Bonus" = 2 total
                    {'fee': '0.00', 'isPricePercentage': False, 'price': '500.00', 'upperBound': '5'},
                    {'fee': '0.00', 'isPricePercentage': False, 'price': '300.00'}
                ]}
            ],
            [
                {'pricingType': 'VOLUME', 'tiers': [
                    {'fee': '0.00', 'isPricePercentage': False, 'price': '2000.00', 'upperBound': '3'},
                    {'fee': '100.00', 'isPricePercentage': False, 'price': '1800.00', 'upperBound': '10'},
                    {'fee': '200.00', 'isPricePercentage': False, 'price': '1600.00'}
                ]},
                {'pricingType': 'VOLUME', 'tiers': [
                    {'fee': '0.00', 'isPricePercentage': False, 'price': '500.00', 'upperBound': '5'},
                    {'fee': '0.00', 'isPricePercentage': False, 'price': '300.00'}
                ]}
            ]
        ]
    }
    
    test_df = pd.DataFrame(test_data)
    
    print("="*80)
    print("🧮 DEMOSTRACIÓN: CONTEO AUTOMÁTICO DE TRANSACCIONES POR MATCH")
    print("="*80)
    
    print("\n📊 DATASET DE PRUEBA:")
    print(f"Total de filas: {len(test_df)}")
    
    # Mostrar el conteo manual para verificación
    print("\n🔍 CONTEO MANUAL (para verificación):")
    manual_counts = {}
    for idx, row in test_df.iterrows():
        match_value = row['match']
        if isinstance(match_value, list):
            for match_name in match_value:
                manual_counts[match_name] = manual_counts.get(match_name, 0) + 1
        else:
            manual_counts[match_value] = manual_counts.get(match_value, 0) + 1
    
    for match_name, count in manual_counts.items():
        print(f"   {match_name}: {count} transacciones")
    
    print("\n" + "="*80)
    print("🚀 EJECUTANDO CALCULATE_REVENUE CON CONTEO AUTOMÁTICO")
    print("="*80)
    
    # Ejecutar la función con debug para ver el conteo automático
    result = calculate_revenue(test_df, amount_column='TOTAL_AMOUNT', debug=True)
    
    print("\n" + "="*80)
    print("📋 RESUMEN DE RESULTADOS POR TIPO DE MATCH")
    print("="*80)
    
    # Agrupar resultados por match para mostrar el resumen
    summary = {}
    for idx, row in result.iterrows():
        match_value = row['match']
        revenue = row['revenue']
        details = row['revenue_calculation_details']
        
        if isinstance(match_value, list):
            match_names = match_value
        else:
            match_names = [match_value]
        
        for match_name in match_names:
            if match_name not in summary:
                summary[match_name] = {
                    'transactions': 0,
                    'total_revenue': 0,
                    'tier_used': None
                }
            summary[match_name]['transactions'] += 1
            
            # Buscar el revenue específico de este match en los detalles
            if details:
                for detail in details:
                    if detail['match_name'] == match_name:
                        summary[match_name]['total_revenue'] += detail['revenue']
                        if 'tier_selected' in detail:
                            summary[match_name]['tier_used'] = detail['tier_selected']
    
    for match_name, data in summary.items():
        print(f"\n📈 {match_name}:")
        print(f"   Transacciones procesadas: {data['transactions']}")
        print(f"   Revenue total: {data['total_revenue']:,.2f}")
        if data['tier_used']:
            print(f"   Tier usado: {data['tier_used']}")
    
    return result


def show_revenue_summary(df_with_revenue: pd.DataFrame):
    """
    Muestra un resumen claro del revenue calculado
    """
    print("\n" + "="*80)
    print("📊 RESUMEN FINAL DE REVENUE")
    print("="*80)
    
    # Resumen general
    total_revenue = df_with_revenue['revenue'].sum()
    transactions_with_revenue = (df_with_revenue['revenue'] > 0).sum()
    total_transactions = len(df_with_revenue)
    
    print(f"💰 Revenue Total: {total_revenue:,.2f}")
    print(f"📈 Transacciones con Revenue: {transactions_with_revenue} de {total_transactions}")
    if transactions_with_revenue > 0:
        print(f"📊 Revenue Promedio: {total_revenue/transactions_with_revenue:,.2f}")
    
    # Top revenue por match
    print(f"\n🏆 TOP MATCHES POR REVENUE:")
    
    # Crear lista de todos los matches individuales con su revenue
    match_revenues = []
    for idx, row in df_with_revenue.iterrows():
        if row['revenue'] > 0 and row['revenue_calculation_details']:
            for detail in row['revenue_calculation_details']:
                match_revenues.append({
                    'match_name': detail['match_name'],
                    'revenue': detail['revenue'],
                    'calculation_method': detail['calculation_method']
                })
    
    if match_revenues:
        match_df = pd.DataFrame(match_revenues)
        top_matches = match_df.groupby(['match_name', 'calculation_method'])['revenue'].agg(['sum', 'count', 'mean']).round(2)
        top_matches = top_matches.sort_values('sum', ascending=False)
        
        for (match_name, calc_method), data in top_matches.head(10).iterrows():
            print(f"   {match_name} ({calc_method})")
            print(f"     └─ Total: {data['sum']:,.2f} | Transacciones: {int(data['count'])} | Promedio: {data['mean']:,.2f}")
    
    return df_with_revenue


def workflow_completo_ejemplo(mm_df, rules_df, amount_column='TOTAL_AMOUNT'):
    """
    Ejemplo completo del workflow: aplicar reglas + calcular revenue
    """
    print("="*80)
    print("🚀 INICIANDO WORKFLOW COMPLETO")
    print("="*80)
    
    # Paso 1: Aplicar reglas a MM
    print("\n📋 PASO 1: Aplicando reglas de matching...")
    df_with_matches = apply_rules_to_mm(mm_df, rules_df, debug=False)
    
    # Ver estadísticas de matches
    total_rows = len(df_with_matches)
    rows_with_matches = df_with_matches['match'].notna().sum()
    multiple_matches = sum(1 for x in df_with_matches['match'] if isinstance(x, list))
    
    print(f"   ✅ Procesadas {total_rows} transacciones")
    print(f"   ✅ {rows_with_matches} transacciones con matches ({rows_with_matches/total_rows*100:.1f}%)")
    print(f"   ✅ {multiple_matches} transacciones con múltiples matches")
    
    # Paso 2: Calcular revenue
    print(f"\n💰 PASO 2: Calculando revenue...")
    df_final = calculate_revenue(df_with_matches, amount_column=amount_column, debug=False)
    
    # Ver estadísticas de revenue
    total_revenue = df_final['revenue'].sum()
    rows_with_revenue = (df_final['revenue'] > 0).sum()
    
    print(f"   ✅ Revenue total calculado: {total_revenue:,.2f}")
    print(f"   ✅ {rows_with_revenue} transacciones generaron revenue")
    
    # Paso 3: Mostrar resumen
    print(f"\n📊 PASO 3: Generando resumen...")
    show_revenue_summary(df_final)
    
    return df_final


def verify_data_structure(mm_df, rules_df):
    """
    Verifica la estructura de los dataframes antes de procesar
    """
    print("🔍 VERIFICANDO ESTRUCTURA DE DATOS")
    print("="*50)
    
    print("\n📋 DATAFRAME MM:")
    print(f"   Filas: {len(mm_df)}")
    print(f"   Columnas: {list(mm_df.columns)}")
    
    print("\n📋 DATAFRAME RULES:")
    print(f"   Filas: {len(rules_df)}")
    print(f"   Columnas: {list(rules_df.columns)}")
    
    if 'PROPERTY_FILTERS' in rules_df.columns:
        print(f"   Reglas con filtros: {rules_df['PROPERTY_FILTERS'].notna().sum()}")
        
    print("\n✅ Estructura verificada")


# ============= CASOS DE USO =============

# Caso 1: Con datos reales
# df_result = apply_rules_to_mm(payouts_mms_df, rules_df, debug=True)
# df_with_revenue = calculate_revenue(df_result, amount_column='TOTAL_AMOUNT', debug=True)

# Caso 2: Probar con datos de ejemplo - LINEAR pricing
# test_result = test_multiple_rules_scenario()

# Caso 3: Probar VOLUME pricing específicamente
# volume_result = test_volume_pricing_scenario()

# Caso 4: Probar pricing mixto (LINEAR + VOLUME)
# mixed_result = test_mixed_pricing_scenario()

# Caso 5: ⭐ Demostrar conteo automático de transacciones por match
# auto_count_result = test_automatic_transaction_counting()

# Caso 6: Workflow completo
# resultado_final = workflow_completo_ejemplo(mm_df, rules_df, 'TOTAL_AMOUNT')

# Caso 7: Verificar estructura de datos
# verify_data_structure(payouts_mms_df, rules_df)

In [538]:
show(test_volume_pricing_scenario(), maxBytes=0)


=== DATOS DE PRUEBA - VOLUME PRICING ===

vol_low:
  Amount: 100,000
  Match: Volume Pricing Basic
  Tiers disponibles:
    Tier 1: hasta 100 tx → 2000.00 (fijo)
    Tier 2: hasta 500 tx → 1900.00 (fijo)
    Tier 3: hasta Sin límite tx → 1700.00 (fijo)

vol_mid:
  Amount: 200,000
  Match: Volume Pricing Basic
  Tiers disponibles:
    Tier 1: hasta 100 tx → 2000.00 (fijo)
    Tier 2: hasta 500 tx → 1900.00 (fijo)
    Tier 3: hasta Sin límite tx → 1700.00 (fijo)

vol_high:
  Amount: 500,000
  Match: Volume Pricing Basic
  Tiers disponibles:
    Tier 1: hasta 100 tx → 2000.00 (fijo)
    Tier 2: hasta 500 tx → 1900.00 (fijo)
    Tier 3: hasta Sin límite tx → 1700.00 (fijo)

vol_percentage:
  Amount: 150,000
  Match: Volume Pricing Percentage
  Tiers disponibles:
    Tier 1: hasta 100 tx → 0.50 (%)
    Tier 2: hasta 500 tx → 0.40 (%)
    Tier 3: hasta Sin límite tx → 0.30 (%)

vol_with_fee:
  Amount: 300,000
  Match: Volume Pricing With Fee
  Tiers disponibles:
    Tier 1: hasta 200 tx → 15

Loading ITables v2.4.0 from the internet... (need help?)


In [540]:
result_df = apply_rules_to_mm(payouts_mms_df.head(222), invoice_pricing_df,debug=False)
show(result_df, maxBytes=0)

Loading ITables v2.4.0 from the internet... (need help?)


In [541]:
show(calculate_revenue(result_df), maxBytes=0)


Loading ITables v2.4.0 from the internet... (need help?)


In [58]:
import pandas as pd

def add_match_flag(df: pd.DataFrame,
                   config: dict,
                   new_col: str = "match_flag",
                   product_name: str = None) -> pd.DataFrame:
    """
    Add a boolean column to `df` that flags rows where all of the columns
    in `config` take one of the allowed values.

    Parameters
    ----------
    df : pd.DataFrame
        The input DataFrame, which must have columns corresponding to the keys
        of `config` (in uppercase form, since the example uses uppercase column names).
    config : dict
        A mapping from lowercase column names to a list of allowed values. E.g.:
            {
              'flow': ['PAYIN'],
              'transaction_type': ['BALANCE RECHARGE', 'DAC_DEBIT', ...]
            }
        Each key in `config` will be converted to uppercase to match the actual df columns.
    new_col : str, optional (default="match_flag")
        The name of the new boolean column that will be added to `df`.  Rows where
        all config‐specified columns match one of their allowed values will be True.

    Returns
    -------
    pd.DataFrame
        The same DataFrame with an extra boolean column `new_col`.
    """

    # Make sure we start with all False
    df[new_col] = False

    # Build a mask for "all conditions met"
    # For each (key, allowed_values), convert key to uppercase to match df column names,
    # then check df[col].isin(allowed_values). Combine all with & (logical AND).
    mask = pd.Series(True, index=df.index)
    for key, allowed_values in config.items():
        search_col = key.upper()
        column_name = product_name + '_' + search_col
        if search_col == 'VALIDATION':
            continue
        if search_col not in df.columns:
            raise KeyError(f"Column '{search_col}' not found in DataFrame.")
        mask &= df[search_col].isin(allowed_values)

    # Assign True to rows that satisfy every condition
    df.loc[mask, column_name] = True

    return df


In [59]:
test_config = invoice_pricing_df[invoice_pricing_df['USAGE_METRIC'] == 'Any Cash-out Transactions']['PROPERTY_FILTERS'].iloc[0]

In [62]:
test_config

{'flow': ['PAYOUT'], 'validation': ['true']}

In [60]:
flagged_df = add_match_flag(df=payouts_mms_df, config=test_config, product_name='ANY_CASH_OUT_TRANSACTIONS')

In [61]:
flagged_df

,MM_ID,CLIENT_ID,EVENTTYPE,EVENTTIMESTAMP,FLOW,TRANSACTION_TYPE,ORIGINATION_SYSTEM,SOURCE_ACCOUNT_TYPE,TRANSFER_METHOD,DESTINATION_BANK,ORIGIN_BANK,COUNTRY,match_flag,ANY_CASH_OUT_TRANSACTIONS_VALIDATION
0,mm_65OFXYLe5P53ns,cli_bdgczxcwd0,COBRE BILLING,2025-05-20,PAYOUT,FAST_PAY,PORTAL,COBRE BALANCE,FAST_PAY,BANCO DE BOGOTA,BANCO DE BOGOTA,COLOMBIA,False,True
1,mm_wvMuSemSPQEICo,cli_bdgczxcwd0,COBRE BILLING,2025-05-20,PAYOUT,FAST_PAY,PORTAL,COBRE BALANCE,FAST_PAY,BANCOLOMBIA,BANCOLOMBIA,COLOMBIA,False,True
2,mm_0vl2COX82CJS0p,cli_bdgczxcwd0,COBRE BILLING,2025-05-20,PAYOUT,FAST_PAY,PORTAL,COBRE BALANCE,FAST_PAY,BANCOLOMBIA,BANCOLOMBIA,COLOMBIA,False,True
3,mm_hPkmRwn50xFkJK,cli_bdgczxcwd0,COBRE BILLING,2025-05-20,PAYOUT,FAST_PAY,PORTAL,COBRE BALANCE,FAST_PAY,BANCOLOMBIA,BANCOLOMBIA,COLOMBIA,False,True
4,mm_UJYnpV9eVGob5w,cli_bdgczxcwd0,COBRE BILLING,2025-05-23,PAYOUT,FAST_PAY,PORTAL,COBRE BALANCE,FAST_PAY,BANCOLOMBIA,BANCOLOMBIA,COLOMBIA,False,True
5,mm_MI9YoLMgmR9zsC,cli_bdgczxcwd0,COBRE BILLING,2025-05-12,PAYOUT,ACH,PORTAL,COBRE BALANCE,ACH,BANCO CAJA SOCIAL BCSC SA,BANCOLOMBIA,COLOMBIA,False,True
6,mm_3JRC06JpQOLQzq,cli_bdgczxcwd0,COBRE BILLING,2025-05-20,PAYOUT,FAST_PAY,PORTAL,COBRE BALANCE,FAST_PAY,BANCOLOMBIA,BANCOLOMBIA,COLOMBIA,False,True
7,mm_lLVeigHqSp9qFa,cli_bdgczxcwd0,COBRE BILLING,2025-05-10,PAYOUT,FAST_PAY,PORTAL,COBRE BALANCE,FAST_PAY,BANCOLOMBIA,BANCOLOMBIA,COLOMBIA,False,True
8,mm_mGb1cF7o95JAQN,cli_bdgczxcwd0,COBRE BILLING,2025-05-10,PAYOUT,FAST_PAY,PORTAL,COBRE BALANCE,FAST_PAY,BANCOLOMBIA,BANCOLOMBIA,COLOMBIA,False,True
9,mm_DyewUMCKuQjT9F,cli_bdgczxcwd0,COBRE BILLING,2025-05-10,PAYOUT,FAST_PAY,PORTAL,COBRE BALANCE,FAST_PAY,BANCOLOMBIA,BANCOLOMBIA,COLOMBIA,False,True
